In [2]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [1]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [5]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [45]:
freq = 4
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])
term = data_source[:Yeld_10Y] .- data_source[:T_Bill] 

data = merge(excess_returns, real_t_bill, dy, term, log1p.(data_source[:T_Bill]))

start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq))


,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1393,0.7471,0.0586,0.2386,-1.026,-0.5885,-0.087,0.1453,0.9413,0.421,0.0773
Price_10Y_T_Bill,0.0851,0.6398,0.0108,0.1091,-0.5762,0.5559,-0.0877,-0.0183,1.491,0.1268,-0.0128
T_Bill_CPI,0.0162,0.1019,0.0059,0.0259,-0.1073,-0.3253,-0.013,0.4055,0.9784,0.3641,0.0092
Dividend_Stock_Price,0.8084,-11.0987,-14.4962,-13.2889,-17.9466,-0.1532,-15.8611,0.9766,-0.9167,-17.9326,-14.1149
Yeld_10Y_T_Bill,0.0246,0.1588,0.0546,0.0898,-0.1224,-0.2701,0.0191,0.8068,0.0302,2.2191,0.0559
T_Bill,0.0586,0.5761,0.1649,0.2223,0.0004,0.742,0.0786,0.9548,0.8548,2.8124,0.1669


nothing

In [14]:
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill,std
Total_Stock_T_Bill,0.1355,0.1217,0.1308,1.1797,0.0277,-0.1808,-0.5434,0.067
Price_10Y_T_Bill,-0.0343,-0.1262,-0.0605,0.9065,-0.0068,0.8016,0.0603,0.0399
T_Bill_CPI,-0.0041,-0.006,0.0021,0.3007,-0.0003,0.072,0.083,0.0071
Dividend_Stock_Price,-0.1046,-0.107,-0.1413,-1.4515,0.9757,0.107,0.3704,0.0693
Yeld_10Y_T_Bill,0.0031,-0.0036,0.0151,-0.0251,0.0004,0.8235,0.014,0.0071
T_Bill,0.0015,0.0188,-0.0061,-0.0897,0.0005,0.0828,0.9791,0.0083


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
Total_Stock_T_Bill,1.0,0.14,0.05,-0.97,0.1,-0.19
Price_10Y_T_Bill,0.14,1.0,0.17,-0.12,-0.03,-0.6
T_Bill_CPI,0.05,0.17,1.0,-0.07,-0.14,0.01
Dividend_Stock_Price,-0.97,-0.12,-0.07,1.0,-0.1,0.18
Yeld_10Y_T_Bill,0.1,-0.03,-0.14,-0.1,1.0,-0.77
T_Bill,-0.19,-0.6,0.01,0.18,-0.77,1.0


In [24]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,-0.0328,0.0091,0.0199,-17.2668,-0.0224,0.1821
5,-0.0067,0.0113,0.0058,-16.9078,0.0218,0.1418
15,0.0183,0.0121,0.0021,-16.4221,0.038,0.1206
25,0.0276,0.0111,0.0014,-16.1709,0.0416,0.1179


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,0.1256,0.0667,0.015,0.2751,0.0226,0.0289
5,0.0588,0.031,0.0089,0.6719,0.0268,0.0552
15,0.0319,0.0142,0.0077,0.9536,0.0178,0.0665
25,0.024,0.0091,0.0078,1.0298,0.0141,0.068


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,-0.0029,0.0029,0.0247,-0.0166,0.0014,0.0084
5,-0.0082,-0.0064,-0.0496,-0.0053,0.0039,-0.0199
15,-0.0136,-0.0056,-0.0333,0.0242,0.0008,-0.0173
25,0.0096,0.0084,-0.0223,0.048,-0.0031,-0.0221


,Total_Stock_T_Bill,Price_10Y_T_Bill,T_Bill_CPI,Dividend_Stock_Price,Yeld_10Y_T_Bill,T_Bill
1,-0.0335,0.0163,-0.0462,0.005,0.0226,-0.0601
5,0.0244,0.0792,0.0096,0.0245,0.1103,0.0885
15,-0.0055,-0.022,0.0754,-0.03,-0.0431,0.0509
25,-0.0151,-0.0698,0.0913,-0.0477,0.0022,0.018


,0.03,0.25,0.5,0.75,0.97
1,-0.2684,-0.1191,-0.0334,0.0529,0.206
5,-0.1175,-0.046,-0.006,0.0319,0.1035
15,-0.0419,-0.0033,0.0182,0.0401,0.0787
25,-0.0175,0.0111,0.0277,0.0439,0.0727


,0.03,0.25,0.5,0.75,0.97
1,-0.1151,-0.0357,0.0089,0.0541,0.1345
5,-0.0474,-0.0094,0.0113,0.0322,0.0696
15,-0.0145,0.0024,0.0122,0.0216,0.0389
25,-0.006,0.0049,0.0111,0.0173,0.0282


,0.03,0.25,0.5,0.75,0.97
1,-0.0085,0.0099,0.0199,0.0299,0.0481
5,-0.0109,-0.0002,0.0058,0.0118,0.0223
15,-0.0125,-0.0029,0.0021,0.0073,0.0166
25,-0.0133,-0.0038,0.0015,0.0066,0.016


,0.03,0.25,0.5,0.75,0.97
1,-17.792,-17.4501,-17.2701,-17.0787,-16.7551
5,-18.1706,-17.3629,-16.908,-16.4492,-15.6194
15,-18.2055,-17.0669,-16.4254,-15.7838,-14.6165
25,-18.1066,-16.8771,-16.1678,-15.494,-14.2133


,0.03,0.25,0.5,0.75,0.97
1,-0.0648,-0.0376,-0.0224,-0.0072,0.0202
5,-0.029,0.004,0.0217,0.0396,0.0724
15,0.0046,0.0259,0.038,0.0502,0.0715
25,0.015,0.0322,0.0416,0.0512,0.0681


,0.03,0.25,0.5,0.75,0.97
1,0.1288,0.1627,0.1822,0.2016,0.236
5,0.0366,0.1051,0.1418,0.179,0.2438
15,-0.0049,0.076,0.1209,0.1659,0.2457
25,-0.0102,0.0723,0.1181,0.1641,0.2452


In [40]:
real_equity_scenarios = reshape(scenarios[1, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)
real_bond_scenarios = reshape(scenarios[2, :, :] .+ scenarios[3, :,:], 1, 101, 10_000)

ret_in_years = cum_returns_in_periods(vcat(real_equity_scenarios, real_bond_scenarios), periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names[1:2], string.(periods))

for a in 1:2
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  



,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-0.0129,0.029
5,-0.0009,0.0171
15,0.0204,0.0142
25,0.0291,0.0126


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,0.1284,0.0714
5,0.059,0.0306
15,0.0286,0.0127
25,0.0195,0.0091


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-0.0026,0.0016
5,-0.0102,-0.0221
15,-0.0145,-0.005
25,-0.0053,0.0209


,Total_Stock_T_Bill,Price_10Y_T_Bill
1,-0.0204,0.0102
5,-0.0027,0.0354
15,-0.0253,-0.0009
25,-0.0622,-0.0204


,0.03,0.25,0.5,0.75,0.97
1,-0.2544,-0.1005,-0.0117,0.0741,0.2295
5,-0.1123,-0.0405,-0.0006,0.0383,0.1082
15,-0.0333,0.0013,0.0205,0.0399,0.074
25,-0.0078,0.0157,0.0291,0.0425,0.0657


,0.03,0.25,0.5,0.75,0.97
1,-0.1045,-0.0183,0.029,0.0769,0.1633
5,-0.0406,-0.0033,0.0172,0.0381,0.0747
15,-0.0098,0.0055,0.0142,0.0226,0.0378
25,-0.0047,0.0063,0.0125,0.0188,0.0297


In [44]:
mdd, mddl = max_drawdown_and_length(real_equity_scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(real_bond_scenarios[1,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.267612  0.363565  0.433302  0.509125  0.653552

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 18.0  30.0  40.0  56.0  90.0

:Price_10Y_T_Bill

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.186707  0.254338  0.308184  0.374712  0.505853

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 21.0  34.0  44.0  59.0  83.0